#### BART
- transformer 기반의 모델 
- Encoder, Decoder 모두 사용하는 모델 
- 장문의 테스트에서 요약 데이터를 생성하는데 사용하는 모델 
- Encoder : 입력이 되는 데이터를 BERT형식으로 문장을 이해 ( 순방향, 역방향 )
- Decoder : 출력이 되는 요약문은 GPT형식으로 문장을 생성 
- 해당 모델에서는 Tokenizer는 Sentencepeice를 이용
- 인풋 tokenizer와 아웃풋 toeknizer는 따로 사용

In [2]:
# !pip install evaluate rouge_score

In [3]:
import numpy as np 
from datasets import Dataset, DatasetDict
# 문장 간의 검증 지표를 만들어주는 라이브러리 
import evaluate
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, \
    DataCollatorForSeq2Seq, Seq2SeqTrainer, \
    Seq2SeqTrainingArguments


In [9]:
# 미리 학습된 모델 선정 (kobart) -> 완벽한 모델 x
model_name = "gogamza/kobart-base-v2"

In [5]:
#  학습에서 사용할 원문 데이터, 요약 데이터 
train_docs = [
    '정부는 중소기업 세제 해택과 R&D 세액 공제를 확대한다고 밝혔다', 
    '해당 기업은 분기 실적에서 매출 성장을 기록했으며 신제품 출시를 예고했다'
]
train_sums = [
    '정부가 중소기업 지원을 확대한다', 
    '기업이 실적 개선과 신제품 출시를 발표했다'
]
valid_docs = [
    '교육부가 디지털 교과서 도입을 추진한다고 발표했다'
]
valid_sums = [
    '교육부가 디지털 교과서 도입을 추진한다.'
]

In [6]:
# transformer 모델에서 list형태의 데이터를 사용x -> Dataset -> DatasetDict
raw_ds = DatasetDict(
    {
        "train" : Dataset.from_dict(
            {
                'document' : train_docs, 
                'summary' : train_sums
            }
        ), 
        'validation' : Dataset.from_dict(
            {
                'document' : valid_docs, 
                'summary' : valid_sums
            }
        )
    }
)

In [7]:
raw_ds

DatasetDict({
    train: Dataset({
        features: ['document', 'summary'],
        num_rows: 2
    })
    validation: Dataset({
        features: ['document', 'summary'],
        num_rows: 1
    })
})

In [11]:
# 토크나이저, 모델을 로드 
# use_fast -> 가속모드 사용 유무 -> kobart 모델에는 RUST기반의 가속모드 존재 
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# 입력 / 출력 문장의 최대 길이 설정 
max_input_len = 512
max_target_len = 128

You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.
You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.
You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


In [12]:
# 데이터 전처리 -> 토큰화 
def tok_fn(batch):
    # batch -> 인자값 -> 묶음형 데이터셋
    # raw_ds의 데이터들의 묶음 -> raw_ds에서 인풋 데이터 -> documet의 데이터
    # 입력 데이터의 토큰화 -> 인코딩
    inputs = tokenizer(
        batch['document'], 
        max_length = max_input_len, 
        padding = 'max_length',      # 고정길이 패딩 사용
        truncation = True            # 최대 길이보다 긴 경우 자른다.
    )
    # 출력 데이터의 토큰화 -> 인코딩 
    # 아웃풋의 토큰나이저는 아웃풋 전용 토크나이저 사용 
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            batch['summary'], 
            max_length = max_target_len, 
            padding = 'max_length',
            truncation = True
        )
    # inputs에서 사용한 토크나이저와 labels에서 사용한 토크나이저는 다른 토크나이저

    # padding 토큰을 -100으로 변경 (CrossEtropyloss에서 -100 이라는 값을 무시)
    # tokenizer -> result는 input_ids, attention_mask, token_type_ids 값을 생성
    # input_ids : 단어 사전에 있는 인덱스의 값들로 인코딩된 데이터 
    # attention_mask : 실제 토큰, 패딩 토큰 
    # token_type_ids : 문장의 위치
    labels_ids = np.array(labels['input_ids'])
    # 행렬에서 pad_token_id와 같은 값을 지닌 데이터를 -100으로 변경
    labels_ids[ labels_ids == tokenizer.pad_token_id ] = -100
    # labels_ids를 labels에 있는 input_ids에 대입 
    inputs['labels'] = labels_ids.tolist()

    return inputs


In [13]:
# tok_fn 호출 
# raw_ds을 이용하여 tok_fn에 대입 

tokenized_ds = raw_ds.map(
    tok_fn, 
    batched= True, 
    remove_columns=['document', 'summary']
)

Map:   0%|          | 0/2 [00:00<?, ? examples/s]c:\Users\ekfla\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\tokenization_utils_base.py:4034: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Map: 100%|██████████| 1/1 [00:00<00:00, 325.97 examples/s]


In [14]:
tokenized_ds

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 2
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 1
    })
})

In [15]:
# DataCollator -> padding 토큰 동적인 처리 + 모델 입력 형태 자동 구성 
data_collator = DataCollatorForSeq2Seq(
    tokenizer= tokenizer, 
    model = model
)

In [16]:
# 검증 지표 선택 
rouge = evaluate.load('rouge')

In [17]:
def compute_metrics(eval_pred):
    preds, labels = eval_pred

    # vocab -> ['A', 'B', 'C']
    # 디코더 -> preds = [13, 17, 102, ...] -> 
    # vocab 인덱스로 구성된 리스트를 다시 문자의 형태로 변환하기 위해서 
    # padding 토큰은 무시하기 위해 -100으로 구성 -> 원래의 padding 토큰의 id 값으로 전환
    labels = np.where(
        labels != -100, labels, tokenizer.pad_token_id
    )
    # 텍스트 디코딩 
    pred_str = tokenizer.batch_decode(preds, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # 문장에서 좌우의 공백이 존재하는 경우 다른 값으로 측정하기때문에 좌우 공백 제거 
    pred_str = [ doc.strip() for doc in pred_str ]
    label_str = [ doc.strip() for doc in label_str ]

    # ROUGE 계산 
    result = rouge.compute(
        predictions= pred_str, 
        references= label_str, 
        use_stemmer = True       # 점수 계산시 단어의 어간을 기준으로 비교하는 의미
    )

    # ROUGE를 보기 편한 형태로 변경 
    result = { k : round(v * 100, 2) for k, v in result.items() }

    return result

In [18]:
# Trainer 파라미터 값을 지정 
args = Seq2SeqTrainingArguments(
    output_dir= "./kobart", 
    eval_strategy= 'epoch', 
    save_strategy= 'epoch', 
    learning_rate= 5e-5, 
    num_train_epochs= 10, 
    logging_steps=10, 

    # generate 설정을 변경 
    predict_with_generate=True, 
    generation_max_length= 70, 
    # 요약 데이터를 생성할때 문장 후보의 탐색의 개수를 설정
    generation_num_beams= 4, 

    load_best_model_at_end= True, 
    metric_for_best_model='rougeL', 
    greater_is_better=True, 
    report_to=[]
)

In [19]:
# Trainer 생성 
trainer = Seq2SeqTrainer(
    model = model, 
    args=args, 
    train_dataset= tokenized_ds['train'], 
    eval_dataset=tokenized_ds['validation'], 
    tokenizer = tokenizer, 
    data_collator= data_collator, 
    compute_metrics= compute_metrics
)
trainer.train()

C:\Users\ekfla\AppData\Local\Temp\ipykernel_24768\780977535.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
c:\Users\ekfla\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,No log,4.365111,0.000000,0.000000,0.000000,0.000000
2,No log,4.355832,0.000000,0.000000,0.000000,0.000000
3,No log,5.281228,0.000000,0.000000,0.000000,0.000000
4,No log,6.177665,0.000000,0.000000,0.000000,0.000000
5,No log,6.384804,0.000000,0.000000,0.000000,0.000000
6,No log,6.322051,0.000000,0.000000,0.000000,0.000000
7,No log,6.630048,0.000000,0.000000,0.000000,0.000000
8,No log,6.963058,0.000000,0.000000,0.000000,0.000000
9,No log,7.028571,0.000000,0.000000,0.000000,0.000000
10,2.623100,6.976777,0.000000,0.000000,0.000000,0.000000


c:\Users\ekfla\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\modeling_utils.py:3918: UserWarning: Moving the following attributes in the config to the generation config: {'forced_eos_token_id': 1}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
c:\Users\ekfla\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
c:\Users\ekfla\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
c:\Users\ekfla\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning

TrainOutput(global_step=10, training_loss=2.6230655670166017, metrics={'train_runtime': 50.7081, 'train_samples_per_second': 0.394, 'train_steps_per_second': 0.197, 'total_flos': 6097364582400.0, 'train_loss': 2.6230655670166017, 'epoch': 10.0})

In [22]:
test_text = """과학기술정보통신부는 초거대 AI 연구 인프라 지원을 강화한다고 밝혔다. 
스타트업 새당으로 GPU 리소스를 확대 제공할 계획이다."""

inputs = tokenizer(
    test_text, 
    return_tensors = 'pt', 
    truncation = True, 
    max_length = max_input_len
)
inputs.pop('token_type_ids', None)

gen_ids = model.generate(
    **inputs.to(model.device), 
    max_length = 20, 
    num_beams = 4, 
    do_sample = False
)

print(tokenizer.decode(gen_ids[0], skip_special_tokens=True))

이동통신부는 초거대 AI 연구 인프라 지원을 강화한다고 밝혔다. 
스타트업 새
